In [1]:
from PIL import Image
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [2]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [3]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [4]:
# options
img_width, img_height = 30, 30
batch_size = 32
epochs = 10

# initializes variable only, gets set later
num_classes=15
sample_size = 15000

data_dir = 'data/food'
resized_dir = '/resized'
original_dir = '/og'

In [5]:
# # resizes images from directory into new subdirectory
# def resize(folder, og_dir, new_dir, fileName, width, height):
#     filePath = os.path.join(folder, fileName)
#     im = Image.open(filePath)
#     newIm = im.resize((int(width), int(height)))
#     directory = re.sub(og_dir, new_dir, folder, count=1) 
    
#     # create directory if it doesn't exist
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
#     # create new file only if it doesn't exist
#     if not os.path.exists(directory + '/' + fileName):
#         newIm.save(directory + '/' + fileName)

# # finds images in directory
# def bulkResize(imageFolder, original_dir, new_dir, width, height):
#     imgExts = ["png", "bmp", "jpg"]
    
# #     # sets number of classes
# #     num_classes = len([name for name in os.listdir(data_dir+original_dir+".")])
#     for path, dirs, files in os.walk(imageFolder + original_dir):
#         for fileName in files:
#             ext = fileName[-3:].lower()
#             if ext not in imgExts:
#                 continue

#             resize(path, original_dir, new_dir, fileName, width, height)

# bulkResize(data_dir, original_dir, resized_dir, img_width, img_height)

In [6]:
# convert image to array and add to df
def loadImgToArray(path, fileName):
    
    # don't count files if file is corrupt
    try:
        im = Image.open(path+'/'+fileName)
        im.mode = "RGB"
        ar = img_to_array(im)
    
        # normalize values
        ar = ar/255.0

        reshape = ar.reshape(img_width*img_height*3)
        batch_array.append(reshape)
        remove = re.sub(data_dir+resized_dir, '', path)
        remove = re.sub(r'\\', '', remove)
        batch_label.append(remove)
    except:
        im.close()


# convert to arrays
def bulkConvert(imageFolder):
    imgExts = ["png", "bmp", "jpg"]
    for path, dirs, files in os.walk(imageFolder):
        for i,  fileName in enumerate(files):
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue

            loadImgToArray(path, fileName)
    
    
batch_array = []
batch_label = []
bulkConvert(data_dir + resized_dir)

In [7]:
# set variables

sample_size = len(batch_array)
num_classes = len(set(batch_label))

In [8]:
print(sample_size)
print(num_classes)

14999
15


In [9]:
feat_cols = [ 'pixel '+str(i+1) for i in range(img_width*img_height*3)]
df = pd.DataFrame(batch_array, columns=feat_cols)
df['label'] = batch_label

In [10]:
df.head()

,pixel 1,pixel 2,pixel 3,pixel 4,pixel 5,pixel 6,pixel 7,pixel 8,pixel 9,pixel 10,...,pixel 2692,pixel 2693,pixel 2694,pixel 2695,pixel 2696,pixel 2697,pixel 2698,pixel 2699,pixel 2700,label
0,0.243137,0.270588,0.243137,0.152941,0.172549,0.145098,0.380392,0.384314,0.352941,0.443137,...,0.184314,0.180392,0.172549,0.188235,0.188235,0.180392,0.168627,0.168627,0.168627,apple_pie
1,0.266667,0.317647,0.623529,0.062745,0.117647,0.396078,0.109804,0.168627,0.400000,0.160784,...,0.792157,0.768627,0.870588,0.901961,0.886275,0.984314,0.870588,0.858824,0.964706,apple_pie
2,0.466667,0.423529,0.298039,0.435294,0.396078,0.258824,0.447059,0.396078,0.262745,0.490196,...,0.058824,0.047059,0.000000,0.262745,0.239216,0.184314,0.243137,0.207843,0.149020,apple_pie
3,1.000000,0.984314,0.984314,1.000000,0.976471,0.968627,0.976471,0.941176,0.913725,0.941176,...,0.152941,0.039216,0.023529,0.164706,0.043137,0.035294,0.196078,0.062745,0.058824,apple_pie
4,0.054902,0.043137,0.086275,0.047059,0.023529,0.062745,0.023529,0.000000,0.019608,0.074510,...,0.050980,0.050980,0.003922,0.039216,0.043137,0.000000,0.152941,0.149020,0.066667,apple_pie


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Columns: 2701 entries, pixel 1 to label
dtypes: float64(2700), object(1)
memory usage: 309.1+ MB


In [12]:
# get position of label for use later
label_pos = len(df.columns)-1

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_result = pca.fit_transform(df[feat_cols].values)

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
df['pca-four'] = pca_result[:,3]
df['pca-five'] = pca_result[:,4]

print(pca.explained_variance_ratio_)

[0.17797401 0.09101871 0.06072222 0.03257488 0.0310613 ]


In [14]:
df.head()

,pixel 1,pixel 2,pixel 3,pixel 4,pixel 5,pixel 6,pixel 7,pixel 8,pixel 9,pixel 10,...,pixel 2697,pixel 2698,pixel 2699,pixel 2700,label,pca-one,pca-two,pca-three,pca-four,pca-five
0,0.243137,0.270588,0.243137,0.152941,0.172549,0.145098,0.380392,0.384314,0.352941,0.443137,...,0.180392,0.168627,0.168627,0.168627,apple_pie,-2.624642,3.234018,5.591384,0.263984,-3.956809
1,0.266667,0.317647,0.623529,0.062745,0.117647,0.396078,0.109804,0.168627,0.400000,0.160784,...,0.984314,0.870588,0.858824,0.964706,apple_pie,2.971496,1.624588,1.489020,-5.094382,-0.901204
2,0.466667,0.423529,0.298039,0.435294,0.396078,0.258824,0.447059,0.396078,0.262745,0.490196,...,0.184314,0.243137,0.207843,0.149020,apple_pie,-0.490823,0.003896,3.685251,-0.946572,0.987895
3,1.000000,0.984314,0.984314,1.000000,0.976471,0.968627,0.976471,0.941176,0.913725,0.941176,...,0.035294,0.196078,0.062745,0.058824,apple_pie,-2.221373,-7.375000,1.393544,-1.557986,1.064506
4,0.054902,0.043137,0.086275,0.047059,0.023529,0.062745,0.023529,0.000000,0.019608,0.074510,...,0.000000,0.152941,0.149020,0.066667,apple_pie,-11.373532,-2.130729,2.903945,0.904578,-2.241513


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier()
average_score = cross_val_score(rf, df.iloc[:, label_pos+1:], df.iloc[:, label_pos], cv=10)

In [16]:
print('Baseline performance is {0:.2f}%'.format(average_score.mean()*100,'%'))

Baseline performance is 12.25%


In [17]:
batch_id = []
stored_value = []
for value in batch_label:
    if value in stored_value:
        batch_id.append(len(stored_value)-1)
    else:
        stored_value.append(value)
        batch_id.append(len(stored_value)-1)

In [18]:
# split data into holdout groups
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(batch_array, batch_id, test_size=0.25)

In [19]:
# Convert to float32
x_train = np.asarray(x_train, dtype=np.float32)
x_test = np.asarray(x_test, dtype=np.float32)

# Convert to int8
y_train = np.asarray(y_train, dtype=np.uint8)
y_test = np.asarray(y_test, dtype=np.uint8)

In [20]:
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11249 train samples
3750 test samples


In [21]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [22]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_width, img_height)
    x_test = x_test.reshape(x_test.shape[0], 3, img_width, img_height)
    input_shape = (3, img_width, img_height)
else:
    x_train = x_train.reshape(x_train.shape[0], img_width, img_height, 3)
    x_test = x_test.reshape(x_test.shape[0], img_width, img_height, 3)
    input_shape = (img_width, img_height, 3)

# CNN model with batch normalization
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('cnn_batch_normalization_model.h5')

Train on 11249 samples, validate on 3750 samples
Epoch 1/10
11249/11249 [==============================] - 6s 572us/step - loss: 2.5810 - acc: 0.1569 - val_loss: 2.7186 - val_acc: 0.1517
Epoch 2/10
11249/11249 [==============================] - 4s 389us/step - loss: 2.3653 - acc: 0.2340 - val_loss: 2.3360 - val_acc: 0.2267
Epoch 3/10
11249/11249 [==============================] - 4s 383us/step - loss: 2.2608 - acc: 0.2666 - val_loss: 2.3909 - val_acc: 0.2437
Epoch 4/10
11249/11249 [==============================] - 4s 384us/step - loss: 2.1699 - acc: 0.3024 - val_loss: 2.0973 - val_acc: 0.3259
Epoch 5/10
11249/11249 [==============================] - 4s 382us/step - loss: 2.0410 - acc: 0.3399 - val_loss: 2.0582 - val_acc: 0.3371
Epoch 6/10
11249/11249 [==============================] - 4s 383us/step - loss: 1.9421 - acc: 0.3756 - val_loss: 2.1302 - val_acc: 0.3125
Epoch 7/10
11249/11249 [==============================] - 4s 382us/step - loss: 1.8338 - acc: 0.4049 - val_loss: 2.0419 - v

In [23]:
prediction = model.predict(x_test).argmax(axis=-1)
actual = y_test.argmax(axis=-1)

num_cross = pd.crosstab(prediction, actual, rownames=['Prediction'], colnames=['Actual'])
num_cross

Actual,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Prediction,,,,,,,,,,,,,,,
0,64,4,57,5,12,5,20,6,29,24,13,9,17,7,22
1,5,120,5,13,8,25,5,12,12,6,17,4,2,5,15
2,23,0,48,0,5,0,8,2,5,9,2,5,6,0,10
3,2,5,2,97,12,22,0,7,9,7,7,7,4,32,4
4,19,27,20,26,68,23,5,8,27,16,23,8,15,30,21
5,3,14,3,10,13,75,0,16,7,11,10,14,18,19,3
6,6,1,2,1,2,0,66,1,10,1,2,0,14,3,6
7,9,11,6,5,7,7,6,143,12,7,6,8,2,7,11
8,27,21,17,12,19,12,15,13,61,23,17,3,22,11,28


In [24]:
# CNN model with batch normalization with different hyperparamters
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size*2,
          epochs=epochs*2,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('cnn_double_hyperparam_batch_normalization_model.h5')

Train on 11249 samples, validate on 3750 samples
Epoch 1/20
11249/11249 [==============================] - 4s 322us/step - loss: 2.6184 - acc: 0.1525 - val_loss: 2.4862 - val_acc: 0.1781
Epoch 2/20
11249/11249 [==============================] - 3s 246us/step - loss: 2.3968 - acc: 0.2217 - val_loss: 2.4375 - val_acc: 0.2485
Epoch 3/20
11249/11249 [==============================] - 3s 268us/step - loss: 2.2694 - acc: 0.2610 - val_loss: 2.1969 - val_acc: 0.2933
Epoch 4/20
11249/11249 [==============================] - 3s 229us/step - loss: 2.1585 - acc: 0.3056 - val_loss: 2.2043 - val_acc: 0.2939
Epoch 5/20
11249/11249 [==============================] - 3s 233us/step - loss: 2.0633 - acc: 0.3333 - val_loss: 2.1643 - val_acc: 0.2971
Epoch 6/20
11249/11249 [==============================] - 3s 233us/step - loss: 1.9543 - acc: 0.3685 - val_loss: 2.1549 - val_acc: 0.3171
Epoch 7/20
11249/11249 [==============================] - 3s 242us/step - loss: 1.8208 - acc: 0.4062 - val_loss: 2.1383 - v

In [25]:
# CNN model with batch normalization with different hyperparamters
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs*5,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('cnn_50epochs_batch_normalization_model.h5')

Train on 11249 samples, validate on 3750 samples
Epoch 1/50
11249/11249 [==============================] - 5s 461us/step - loss: 2.6202 - acc: 0.1456 - val_loss: 2.4965 - val_acc: 0.1955
Epoch 2/50
11249/11249 [==============================] - 4s 376us/step - loss: 2.3948 - acc: 0.2270 - val_loss: 2.3249 - val_acc: 0.2565
Epoch 3/50
11249/11249 [==============================] - 4s 371us/step - loss: 2.2949 - acc: 0.2595 - val_loss: 2.2488 - val_acc: 0.2736
Epoch 4/50
11249/11249 [==============================] - 4s 386us/step - loss: 2.2037 - acc: 0.2872 - val_loss: 2.2175 - val_acc: 0.2875
Epoch 5/50
11249/11249 [==============================] - 5s 401us/step - loss: 2.1261 - acc: 0.3148 - val_loss: 2.2160 - val_acc: 0.2880
Epoch 6/50
11249/11249 [==============================] - 4s 384us/step - loss: 2.0232 - acc: 0.3440 - val_loss: 2.1008 - val_acc: 0.3197
Epoch 7/50
11249/11249 [==============================] - 4s 369us/step - loss: 1.9316 - acc: 0.3703 - val_loss: 2.4422 - v

In [26]:
# set up dictionary to generate crosstab
dic = {key:value for key, value in zip(list(set(batch_id)), stored_value)}
dic

{0: 'apple_pie',
 1: 'baby_back_ribs',
 2: 'baklava',
 3: 'beef_carpaccio',
 4: 'beef_tartare',
 5: 'beet_salad',
 6: 'beignets',
 7: 'bibimbap',
 8: 'bread_pudding',
 9: 'breakfast_burrito',
 10: 'bruschetta',
 11: 'caesar_salad',
 12: 'cannoli',
 13: 'caprese_salad',
 14: 'carrot_cake'}

In [27]:
# pull prediction and actual values
prediction = model.predict(x_test).argmax(axis=-1)
actual = y_test.argmax(axis=-1)

In [28]:
# convert id values into string values
prediction_string = []
actual_string = []

for value in prediction:
    prediction_string.append(dic[value])

for value in actual:
    actual_string.append(dic[value])

In [29]:
# create categorical variables to input into crosstab
prediction = pd.Categorical(prediction_string, categories=stored_value)
actual = pd.Categorical(actual_string, categories=stored_value)

num_cross = pd.crosstab(prediction, actual, rownames=['Prediction'], colnames=['Actual'])
num_cross

Actual,apple_pie,baby_back_ribs,baklava,beef_carpaccio,beef_tartare,beet_salad,beignets,bibimbap,bread_pudding,breakfast_burrito,bruschetta,caesar_salad,cannoli,caprese_salad,carrot_cake
Prediction,,,,,,,,,,,,,,,
apple_pie,54,6,23,7,10,5,14,10,24,23,9,3,15,6,17
baby_back_ribs,7,121,6,11,11,25,4,20,15,4,15,5,11,9,13
baklava,20,2,57,1,6,3,4,2,9,15,3,8,8,0,11
beef_carpaccio,1,6,3,108,18,17,0,4,5,5,8,4,2,25,3
beef_tartare,20,16,12,21,66,17,4,3,24,13,26,8,15,22,20
beet_salad,5,18,5,13,10,67,1,18,4,7,15,10,5,21,6
beignets,15,5,8,0,8,2,101,7,14,13,6,3,30,3,18
bibimbap,10,11,13,5,9,11,8,152,16,11,6,16,14,15,13
bread_pudding,30,12,15,6,18,11,13,5,37,19,12,10,18,5,22
